In [7]:
import os
from IPython.display import Image
from textwrap import dedent

In [8]:
import random
from typing import Dict, List
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
# import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from peft import (
    LoraConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer

In [9]:
seed = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
seed_everything(seed)

In [10]:
Image(url='https://www.mlexpert.io/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fbanner.2336875a.png&w=3840&q=75', width=500)

## constants

In [11]:
pad_token = "<|pad|>"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "Llama-3-8B-Instruct-Finance-RAG"

## model & tokenzier

In [13]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
print(tokenizer.chat_template)

{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>

' }}{% endif %}


In [18]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8b",
    torch_dtype=torch.float16,
    device_map="auto",
)


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ba/83/ba837357c37c4c572f89ebcdffc2867477dbb4768779d332eb303f848c60d0de/d9eee5f23d94405d90b7e9ff88b9443fee42f8528a658f54214c2aba7530d80c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1734427017&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDQyNzAxN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2JhLzgzL2JhODM3MzU3YzM3YzRjNTcyZjg5ZWJjZGZmYzI4Njc0NzdkYmI0NzY4Nzc5ZDMzMmViMzAzZjg0OGM2MGQwZGUvZDllZWU1ZjIzZDk0NDA1ZDkwYjdlOWZmODhiOTQ0M2ZlZTQyZjg1MjhhNjU4ZjU0MjE0YzJhYmE3NTMwZDgwYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=cmdwgsfjDDvEtLMdFpfQWTLaBtuvhtHexedxTxYFcRnclL-Ap2uhR%7E%7EO2jMuj94m4Jhx4n7e91ascGyUnloLVUEOJPZ7BI7q28aqwwy-jMcAkyJiJEqxiSNA3lO58yI6WFRqbUs9gRO85AVQbtacK3ZYT0E23XtgaiezyVGWcO2PsJudN4Ud7jlq9fqo8hxY6TQAZnjuDbngZrAdv43N42H%7E-9gGRG2b

model-00002-of-00004.safetensors:  81%|########  | 4.05G/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ba/83/ba837357c37c4c572f89ebcdffc2867477dbb4768779d332eb303f848c60d0de/d9eee5f23d94405d90b7e9ff88b9443fee42f8528a658f54214c2aba7530d80c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1734427017&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDQyNzAxN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2JhLzgzL2JhODM3MzU3YzM3YzRjNTcyZjg5ZWJjZGZmYzI4Njc0NzdkYmI0NzY4Nzc5ZDMzMmViMzAzZjg0OGM2MGQwZGUvZDllZWU1ZjIzZDk0NDA1ZDkwYjdlOWZmODhiOTQ0M2ZlZTQyZjg1MjhhNjU4ZjU0MjE0YzJhYmE3NTMwZDgwYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=cmdwgsfjDDvEtLMdFpfQWTLaBtuvhtHexedxTxYFcRnclL-Ap2uhR%7E%7EO2jMuj94m4Jhx4n7e91ascGyUnloLVUEOJPZ7BI7q28aqwwy-jMcAkyJiJEqxiSNA3lO58yI6WFRqbUs9gRO85AVQbtacK3ZYT0E23XtgaiezyVGWcO2PsJudN4Ud7jlq9fqo8hxY6TQAZnjuDbngZrAdv43N42H%7E-9gGRG2b

ConnectionError: (ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 92a1d6c6-5eb0-4608-860d-e49483cb760b)')

In [ ]:
model